In [1]:
#### import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
import seaborn as sns
!pip install fpdf
from fpdf import FPDF

pribor_dic = {1:'Анализатор Ilab Taurus, Ilab 650 ', 2: 'Анализаторы Miura 200/300/One', 7: 'Анализатор Sapphire 400', 11: 'Анализаторы А 25, A 15', 5: 'Анализатор BA-400'}

analit_dic = pd.read_excel(r'C:/Users/ZhuravlevaR/MLPP/analit.xlsx')
analit_dic.index = analit_dic.iloc[:,0]


df = pd.read_excel(r'C:/Users/ZhuravlevaR/MLPP/Z_score.xlsx', sheet_name = 'c9')
analit = []

df.columns
dic = {}
for name in df.columns[3:]:
    if 'Unnamed' not in name:
        tmp = name
        analit += [tmp]
    dic.update({name : tmp + '_' + df.at[0, name]})
df = df.rename(columns = dic)
df = df.iloc[1:]
df.reset_index(drop = True, inplace =True)
len(analit)


df_low = df.copy()
for col in df_low.columns:
    if 'Lev' in col:
        for i in range(0,df_low.shape[0]):
            if 'е' in str(df_low.at[i, col]):
                print(df_low.at[i, col])
                df_low.at[i, col] = None
                df.at[i, col] = float(df.at[i, col][1:])

std_all = df_low.std()
std_cons = std_all.copy()
mean_all = df_low.mean()
mean_cons = mean_all.copy()
count_all = df_low.count()
count_cons = count_all.copy()
for col in mean_all.index:
    tmp = df_low[col][abs(df_low[col]-mean_all[col]) <= 2*std_all[col]].mean()
    tmp2 = df_low[col][abs(df_low[col]-mean_all[col]) <= 2*std_all[col]].std()
    tmp3 = df_low[col][abs(df_low[col]-mean_all[col]) <= 2*std_all[col]].count()
    mean_cons[col] = tmp
    std_cons[col] = tmp2
    count_cons[col] = tmp3
    

res = pd.read_excel(r'C:/Users/ZhuravlevaR/MLPP/Res_protokol_2020_2.xlsx', index_col='Unnamed: 0', sheet_name = 'c9')

# res = pd.DataFrame([], columns=std_all.index)
# res.loc['mean_all'] = mean_all
# res.loc['std_all'] = std_all
# res.loc['mean_cons'] = mean_cons
# res.loc['std_cons'] = std_cons  
# res.loc['count_cons'] = count_cons
# res.loc['count'] = df_low.count()

# for anal in analit:
#     for level in range(1,3):
#         for pribor in [1, 2, 5, 7, 11]:
#             col = anal + '_' + 'Lev'+ str(level)
#             res.loc['count' + '_' + str(pribor),col] = df_low[col][df_low[anal + '_' + 'Pribor'] == pribor].count()
#             mean = df_low[col][df_low[anal + '_' + 'Pribor'] == pribor].mean()
#             std = df_low[col][df_low[anal + '_' + 'Pribor'] == pribor].std()
#             res.loc['mean' + '_' + str(pribor),col] = mean
#             res.loc['std' + '_' + str(pribor),col] = std
#             tmp = df_low[(df_low[anal + '_' + 'Pribor'] == pribor) ]
#             tmp = tmp[(abs(tmp[col]-mean) <= 2 * std)]
#             res.loc['count' + '_' + str(pribor)+ '_cons',col] = tmp[col].count()
#             res.loc['mean' + '_' + str(pribor)+ '_cons',col] = tmp[col].mean()
#             res.loc['std' + '_' + str(pribor)+ '_cons',col] = tmp[col].std()
#             res.loc['CV' + '_' + str(pribor)+ '_cons',col] = tmp[col].std() / tmp[col].mean() * 100
            
lev_col = []
for col in df.columns:
    if 'Lev' in col:
        lev_col +=[col]
df['count_result'] = df[lev_col].count(axis=1)


res.fillna(0, inplace = True)
def roun(floa, i):
    if i :
        return round(floa, i)
    else :
        return int(round(floa))
            
def title(number):
    pdf.image(r'C:/Users/ZhuravlevaR/MLPP/Logotip.png', x=245, y=2, w=50)
    print('№ = {}, Код лаборатории {}'.format(number, df.Code[number]))

    pdf.set_xy(30,10)
    pdf.write(15,'Межлабораторная программа-практикум по биохимическим исследованиям')

    pdf.set_font('DejaVuB', '', 14)
    pdf.set_xy(120,13)
    pdf.write(25,'Цикл 2 – 2020')


    pdf.set_font('DejaVu', '', 10)

    pdf.set_xy(10,18)
    pdf.write(25,'Код лаборатории')

    pdf.set_xy(45,27)
    pdf.set_fill_color(240, 230, 140)
    pdf.cell(15,8,df.Code[number],1,0,'C',1)

    pdf.set_xy(245,25)
    pdf.write(35,'октябрь 2020')

    pdf.set_xy(10,25)
    pdf.write(35,'ЛПУ')
    pdf.set_xy(25,38)
    pdf.set_fill_color(222, 222, 222)
    pdf.cell(140,8,df.Name[number],0,0,'L',1)
    return 0 

def header(height):
    pdf.set_font('DejaVu', '', 10)
    pdf.set_xy(10,height)
    pdf.set_fill_color(245, 245, 245)
    pdf.cell(36,18,'Аналит, метод',1,0,'C',1)
    pdf.set_xy(46,height)
    pdf.set_fill_color(245, 245, 245)
    pdf.cell(14,18,'Проба',1,0,'C',1)

    pdf.set_xy(60,height)
    pdf.set_fill_color(222, 222, 222)
    pdf.multi_cell(20,4.5,'               Ваш   результат        ',1,'C',1)

    pdf.set_xy(80,height)
    pdf.set_fill_color(245, 245, 245)
    pdf.cell(54,18,'Группа сравнения',1,0,'C',1)
    pdf.set_xy(134,height)
    pdf.set_fill_color(245, 245, 245)
    pdf.cell(15,18,'Число',1,0,'C',1)

    pdf.set_xy(149,height)
    pdf.set_fill_color(222, 222, 222)
    pdf.multi_cell(20,6,'Целевое значение (ЦЗ)',1,'C',1)
    pdf.set_xy(169,height)
    pdf.set_fill_color(245, 245, 245)
    pdf.multi_cell(26,6,'Средне- квадратическое отклонение',1,'C',1)

    pdf.set_xy(195,height)
    pdf.set_fill_color(245, 245, 245)
    pdf.multi_cell(32,4.5,'Ваше отклонение относительно ЦЗ',1,'C',1)
    pdf.set_xy(195,height+9)
    pdf.set_fill_color(222, 222, 222)
    pdf.cell(17,9,'%',1,0,'C',1)
    pdf.set_xy(212,height+9)
    pdf.set_fill_color(245, 245, 245)
    pdf.cell(17,9,'SDI',1,0,'C',1)

    pdf.set_xy(227,height)
    pdf.set_fill_color(245, 245, 245)
    pdf.multi_cell(58,4.5,'Допустимое отклонение по ГОСТ Р 53133.1-2008, %',1,'C',1)
    pdf.set_xy(227,height+9)
    pdf.set_fill_color(245, 245, 245)
    pdf.cell(29,9,'рекомендуемое  ',1,0,'C',1)
    pdf.set_xy(256,height+9)
    pdf.set_fill_color(245, 245, 245)
    pdf.multi_cell(29,4.5,'биологически  обоснованное ',1,'C',1)
    return height + 18

def cell(X, analit):
    if pd.isnull(df.at[number,anal + '_Lev1']) and pd.isnull(df.at[number,anal + '_Lev2']):
        return X
    H = 18 

    if df.at[number,anal + '_Pribor'] in [1, 2, 7, 5, 11]:# переделать список в форму
        H = 36 
    if H + X > end_page :
        pdf.add_page()
        X = header(10)
  # аналит, метод
    pdf.set_xy(10,X)
    len_analit = len(analit_dic.loc['rus',analit])//20 + 1
    pdf.cell(36,H,'',1,0,'C') #печатаю пустую ячейку
    pdf.set_xy(10, X + H/2 - len_analit * 2.25)
    pdf.multi_cell(36, 4.5, analit_dic.loc['rus',analit], 0, 'C')
  #old  pdf.multi_cell(36, H / len_analit, analit_dic.loc['rus',analit], 1, 'C')
  # проба
    pdf.set_xy(46,X)
    pdf.cell(14,H/2,'1',1,0,'C')
    pdf.set_xy(46,X+H/2)
    pdf.cell(14,H/2,'2',1,0,'C')
  # Ваш результат
    pdf.set_xy(60,X)
    pdf.set_fill_color(222, 222, 222)
    pdf.cell(20,H/2,str(df.at[number,anal + '_Lev1']),1,0,'C',1)
    pdf.set_xy(60,X+H/2)
    pdf.set_fill_color(222, 222, 222)
    pdf.cell(20,H/2,str(df.at[number,anal + '_Lev2']),1,0,'C',1)
  # Группа сравнения
    pdf.set_xy(80,X)
    pdf.cell(54,9,'Все участники',1,0,'C')
    pdf.set_xy(80,X+H/2)
    pdf.cell(54,9,'Все участники',1,0,'C')
  # число
    pdf.set_xy(134,X)
    pdf.cell(15,9,str(int(res.at['count',anal + '_Lev1'])),1,0,'C')
 
    pdf.set_xy(134,X+H/2)
    pdf.cell(15,9,str(int(res.at['count',anal + '_Lev2'])),1,0,'C')
 
  # Целевое значение
    pdf.set_xy(149,X)
    pdf.set_fill_color(222, 222, 222)
    pdf.cell(20,9,str(roun(res.at['mean_cons',anal + '_Lev1'],analit_dic.at['round1',anal])),1,0,'C',1)
  
    pdf.set_xy(149,X+H/2)
    pdf.set_fill_color(222, 222, 222)
    pdf.cell(20,9,str(roun(res.at['mean_cons',anal + '_Lev2'],analit_dic.at['round2',anal])),1,0,'C',1)
  
  # Среднеквадратическое отклонение
    pdf.set_xy(169,X)
    pdf.cell(26,9,str(roun(res.at['std_cons',anal + '_Lev1'],analit_dic.at['round1',anal])),1,0,'C')
 
    pdf.set_xy(169,X+H/2)
    pdf.cell(26,9,str(roun(res.at['std_cons',anal + '_Lev2'],analit_dic.at['round2',anal])),1,0,'C')
  
  # Смещение %
    pdf.set_xy(195,X)
    pdf.set_fill_color(222, 222, 222)
    if abs(roun((df.at[number,anal + '_Lev1']-res.at['mean_cons',anal + '_Lev1'])*100/res.at['mean_cons',anal + '_Lev1'], 1)) > analit_dic.loc['rec',analit]:
        E = '**'
    else:
        E = ''
    pdf.cell(17,9,str(roun((df.at[number,anal + '_Lev1']-res.at['mean_cons',anal + '_Lev1'])*100/res.at['mean_cons',anal + '_Lev1'], 1)) + E,1,0,'C',1)
    if abs(roun((df.at[number,anal + '_Lev2']-res.at['mean_cons',anal + '_Lev2'])*100/res.at['mean_cons',anal + '_Lev2'], 1)) > analit_dic.loc['rec',analit]:
        F = '**'
    else:
        F = ''
    pdf.set_xy(195,X+H/2)
    pdf.set_fill_color(222, 222, 222)
    pdf.cell(17,9,str(roun((df.at[number,anal + '_Lev2']-res.at['mean_cons',anal + '_Lev2'])*100/res.at['mean_cons',anal + '_Lev2'], 1)) + F,1,0,'C',1)
  
  # SDI
    pdf.set_xy(212,X)
    if roun((df.at[number,anal + '_Lev1']-res.at['mean_cons',anal + '_Lev1'])/res.at['std_cons',anal + '_Lev1'], 1) > 2 or roun((df.at[number,anal + '_Lev1']-res.at['mean_cons',anal + '_Lev1'])/res.at['std_cons',anal + '_Lev1'],1) < -2:
        G = '**'
    else:
        G = ''
    pdf.cell(15,9,str(roun((df.at[number,anal + '_Lev1']-res.at['mean_cons',anal + '_Lev1'])/res.at['std_cons',anal + '_Lev1'], 1)) + G,1,0,'C')
    if roun((df.at[number,anal + '_Lev2']-res.at['mean_cons',anal + '_Lev2'])/res.at['std_cons',anal + '_Lev2'], 1) > 2 or roun((df.at[number,anal + '_Lev2']-res.at['mean_cons',anal + '_Lev2'])/res.at['std_cons',anal + '_Lev2'], 1) < -2:
        J = '**'
    else:
        J = ''
    pdf.set_xy(212,X+H/2)
    pdf.cell(15,9,str(roun((df.at[number,anal + '_Lev2']-res.at['mean_cons',anal + '_Lev2'])/res.at['std_cons',anal + '_Lev2'], 1)) + J,1,0,'C')
  
  # Допустимые
    pdf.set_xy(227,X)
    pdf.cell(29,H,str(analit_dic.loc['rec',analit]),1,0,'C')
    pdf.set_xy(256,X)
    pdf.cell(29,H,str(analit_dic.loc['biol',analit]),1,0,'C')

    if df.at[number,anal + '_Pribor'] in [1, 2, 7, 5, 11] and res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'] > 0:
        pdf.set_xy(80,X+H/4)
        pdf.cell(54,H/4,str(pribor_dic[df.at[number,anal + '_Pribor']]),1,0,'C') 
        
        pdf.set_xy(80,X+H*3/4)
        pdf.cell(54,H/4,str(pribor_dic[df.at[number,anal + '_Pribor']]),1,0,'C')
        
        pdf.set_xy(134,X+H/4)
        pdf.cell(15,H/4,str(int(res.at['count_'+ str(df.at[number,anal + '_Pribor']),anal + '_Lev1'])),1,0,'C')
        
        pdf.set_xy(134,X+H*3/4)
        pdf.cell(15,H/4,str(int(res.at['count_'+ str(df.at[number,anal + '_Pribor']),anal + '_Lev2'])),1,0,'C')
        
        pdf.set_xy(149,X+H/4)
        pdf.set_fill_color(222, 222, 222)
        pdf.cell(20,H/4,str(roun(res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'],analit_dic.at['round1',anal])),1,0,'C',1)
        
        pdf.set_xy(149,X+H*3/4)
        pdf.set_fill_color(222, 222, 222)
        pdf.cell(20,H/4,str(roun(res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'],analit_dic.at['round2',anal])),1,0,'C',1)
        
        pdf.set_xy(169,X+H/4)
        pdf.cell(26,H/4,str(roun(res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'],analit_dic.at['round1',anal])),1,0,'C')
        
        pdf.set_xy(169,X+H*3/4)
        pdf.cell(26,H/4,str(roun(res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'],analit_dic.at['round2',anal])),1,0,'C')
        
        pdf.set_xy(195,X+H/4)
        pdf.set_fill_color(222, 222, 222)
        if abs(roun((df.at[number,anal + '_Lev1']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'])*100/res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'], 1)) >analit_dic.loc['rec',analit]:
            A = '**'
        else:
            A = ''
        pdf.cell(17,H/4,str(roun((df.at[number,anal + '_Lev1']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'])*100/res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'], 1)) + A,1,0,'C',1)
        
        pdf.set_xy(195,X+H*3/4)
        pdf.set_fill_color(222, 222, 222)
        if abs(roun((df.at[number,anal + '_Lev2']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'])*100/res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'], 1)) > analit_dic.loc['rec',analit]:
            D = '**'
        else:
            D = ''
        pdf.cell(17,H/4,str(roun((df.at[number,anal + '_Lev2']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'])*100/res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'], 1)) + D,1,0,'C',1)
        
        pdf.set_xy(212,X+H/4)
        if (df.at[number,anal + '_Lev1']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'])/res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'] > 2 or (df.at[number,anal + '_Lev1']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'])/res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'] < -2:
            B = '**'
        else:
            B = ''
        pdf.cell(15,H/4,str(roun((df.at[number,anal + '_Lev1']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'])/res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1'], 1)) + B,1,0,'C')
        
        pdf.set_xy(212,X+H*3/4)
        if roun((df.at[number,anal + '_Lev2']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'])/res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'], 1) > 2 or roun((df.at[number,anal + '_Lev2']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'])/res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'], 1) < -2:
            C = '**'
        else:
            C = ''
        pdf.cell(15,H/4,str(roun((df.at[number,anal + '_Lev2']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'])/(res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2']), 1)) + C,1,0,'C')    
        #pdf.cell(15,H/4,str(roun((df.at[number,anal + '_Lev2']-res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'])/res.at['std_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev2'], 1)) + C,1,0,'C')

    return X + H

#строка отдельно для липазы, выводится только уровень 2
def lyp(X):
    if pd.isnull(df.at[number,'Lyp' + '_Lev2']):
        return X
    else:
        pdf.set_xy(10,X)
        pdf.cell(36,10,'Липаза, Е/л',1,0,'C')
        pdf.set_xy(46,X)
        pdf.cell(14,10,'2',1,0,'C')
        
        pdf.set_xy(60,X)
        pdf.set_fill_color(222, 222, 222)
        pdf.cell(20,10,str(df.at[number,'Lyp' + '_Lev2']),1,0,'C',1)
        
        pdf.set_xy(80,X)
        pdf.cell(54,10,'Все участники',1,0,'C')
        
        pdf.set_xy(134,X)
        pdf.cell(15,10,str(int(res.at['count','Lyp' + '_Lev2'])),1,0,'C')
        
        pdf.set_xy(149,X)
        pdf.set_fill_color(222, 222, 222)
        pdf.cell(20,10,str(roun(res.at['mean_cons','Lyp' + '_Lev2'],analit_dic.at['round2','Lyp'])),1,0,'C',1)
        
        pdf.set_xy(169,X)
        pdf.cell(26,10,str(roun(res.at['std_cons','Lyp' + '_Lev2'],analit_dic.at['round2','Lyp'])),1,0,'C')
        
        if abs(roun((df.at[number,'Lyp' + '_Lev2']-res.at['mean_cons','Lyp' + '_Lev2'])*100/res.at['mean_cons','Lyp' + '_Lev2'], 1)) > analit_dic.loc['rec','Lyp']:
            F = '**'
        else:
            F = ''
        pdf.set_xy(195,X)
        pdf.set_fill_color(222, 222, 222)
        pdf.cell(17,10,str(roun((df.at[number,'Lyp' + '_Lev2']-res.at['mean_cons','Lyp' + '_Lev2'])*100/res.at['mean_cons','Lyp' + '_Lev2'], 1)) + F,1,0,'C',1)
        
        if roun((df.at[number,'Lyp' + '_Lev2']-res.at['mean_cons','Lyp' + '_Lev2'])/res.at['std_cons','Lyp' + '_Lev2'], 1) > 2 or roun((df.at[number,'Lyp' + '_Lev2']-res.at['mean_cons','Lyp' + '_Lev2'])/res.at['std_cons','Lyp' + '_Lev2'], 1) < -2:
            J = '**'
        else:
            J = ''
        pdf.set_xy(212,X)
        pdf.cell(15,10,str(roun((df.at[number,'Lyp' + '_Lev2']-res.at['mean_cons','Lyp' + '_Lev2'])/res.at['std_cons','Lyp' + '_Lev2'], 1)) + J,1,0,'C')

        pdf.set_xy(227,X)
        pdf.cell(29,10,str(analit_dic.loc['rec','Lyp']),1,0,'C')
        pdf.set_xy(256,X)
        pdf.cell(29,10,str(analit_dic.loc['biol','Lyp']),1,0,'C')
  
    
def end_of_page(X):
    text = """
Целевое значение (ЦЗ) - метод-зависимое (или метод- и прибор-зависимое) робастное среднее результатов, полученных участниками данного цикла. Среднеквадратическое отклонение (S) характеризует разброс результатов вокруг средней величины. 
Отклонение вашего результата (X) от ЦЗ представлено в виде: 
1) отличия вашего результата (X) от ЦЗ, % : (X - ЦЗ)*100 / ЦЗ 
(в качестве  допустимых пределов использованы фиксированные нормы точности для каждого аналита);

2) индекса среднеквадратического отклонения (SDI), отражающего положение вашего результата по отношению к остальным результатам в группе сравнения: SDI = (Х - ЦЗ) / S
(допустимые значения SDI лежат в пределах от -2,0 до 2,0).
SDI не вычислялся, если в группе сравнения было менее 8 результатов.

** Отклонения, выходящие за допустимые пределы, отмечены звёздочками.
                                                                                                
                                                                                                Благодарим за участие, желаем успехов!
    """
    pdf.set_xy(10,X+7)
    pdf.multi_cell(270,4.5,text,0,'L')
     





In [63]:
str(roun((df.at[number,'Lyp' + '_Lev2']-res.at['mean_cons','Lyp' + '_Lev2'])*100/res.at['mean_cons','Lyp' + '_Lev2'], 1))

'-44.8'

In [59]:
res.at['mean_cons','Lyp' + '_Lev2']

46.0

In [38]:
res.at['mean_'+ str(df.at[number,anal + '_Pribor']) + '_cons',anal + '_Lev1']

37.3

In [2]:
for number in range(60,61):
    if df.at[number, 'count_result'] > 0:
        pdf = FPDF(orientation='L')
        pdf.add_page()
        end_page = 185
    # ШАПКА_Протокол 1
        pdf.add_font('DejaVuB', '', r'C:/Users/ZhuravlevaR/MLPP/FreeSerifBold.ttf', uni=True)
        pdf.add_font('DejaVu', '', r'C:/Users/ZhuravlevaR/MLPP/FreeSerif.ttf', uni=True)
        pdf.set_font('DejaVuB', '', 16)

    #print(number)
        title(number) # paint title
        X = header(50)
        for anal in ['ALT', 'Alb', 'Amy', 'AmyP', 'AST', 'Prot', 'DPD', 'BilE', 'BilK', \
                 'GGT', 'GOD', 'HEK', 'Fe', 'K', 'Kd', 'Knd', 'Nad', 'Nand', 'ARS', \
                 'Crea', 'Comp', 'PAP','CFK', 'LDH', 'Mg', 'UA', 'Urea', 'TG', 'P', 'Cl', \
                 'Clor', 'Clnd', 'Chol', 'HDl', 'HDlos', 'LDL', 'ALP', 'ALPIFCC',  'IgA', \
                 'IgM', 'IgG']:
            X = cell(X, anal)
      #breack
        lyp(X)
        end_of_page(X)

        pdf.output(r"C:/Users/ZhuravlevaR/MLPP/MLPP_2020_2_Z/2020_2_{}_Протокол1.pdf".format(df.Code[number]), 'F')
    #df.shape[0]

№ = 60, Код лаборатории Е2


In [34]:
df.at[0, 'count_result']

48